In [ ]:
!pip install langchain_openai langchain_core langchain_community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 57.9 MB/s eta 0:00:00


In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("container-wrapper"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://cesar.breezy.hr/p/00f79174d8ad-pesquisador-em-inteligencia-artificial-e-sistemas-distribuidos",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

4301

### Dividindo Documentos - Splitting/Chunking

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=500, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

6

In [ ]:
print(all_splits[3].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Indexando - Store

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("precisa de doutorado para a vaga?")

len(retrieved_docs)

6

In [ ]:
print(retrieved_docs[0].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Buscando e Recuperando Informações - Retrieve

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


### Gerando Respostas - Generate

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("Tem plano de saúde como benefício?"):
    print(chunk, end="", flush=True)

Sim, o CESAR oferece plano de saúde como um dos benefícios para os colaboradores. Além disso, há também plano odontológico e outros auxílios.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

#### Texto para reprodução


Juliano Moreira


Resumo Profissional
Cientista de Dados com mais de 5 anos de experiência em análise de dados, machine learning e inteligência artificial. Expertise em transformar grandes volumes de dados em insights estratégicos para tomadas de decisão de negócios. Proficiente em Python, SQL, R, e ferramentas de visualização como Power BI e Tableau. Experiência em trabalhar com modelos preditivos, análise estatística e otimização de processos empresariais. Com forte habilidade de comunicação e apresentação, colaborando estreitamente com equipes multifuncionais para implementar soluções baseadas em dados.


Experiência Profissional
Cientista de Dados Sênior
Empresa XYZ — São Paulo, SP
Março 2022 – Atual

Desenvolvimento e implementação de modelos preditivos de aprendizado de máquina para previsão de demanda e otimização de inventário, resultando em uma redução de 15% nos custos operacionais.
Análise de grandes volumes de dados em tempo real para detectar padrões e fornecer insights para equipes de marketing e vendas.
Aplicação de técnicas de NLP (Processamento de Linguagem Natural) para análise de sentimentos em feedbacks de clientes, identificando áreas chave para melhoria de produtos.
Criação de dashboards interativos e relatórios de performance para stakeholders, utilizando Power BI e Tableau.
Liderança de uma equipe de 3 analistas de dados, promovendo o desenvolvimento contínuo e a integração de novas tecnologias.


Cientista de Dados
Tech Solutions Ltda. — São Paulo, SP
Julho 2018 – Fevereiro 2022

Criação de modelos de machine learning para previsão de churn de clientes, com taxa de sucesso de 80% na antecipação de perdas de clientes.
Análise estatística de dados históricos de vendas para identificar tendências e oferecer recomendações de estratégia para a diretoria de vendas.
Desenvolvimento de algoritmos de recomendação para melhorar a experiência do cliente no e-commerce, resultando em um aumento de 10% na taxa de conversão.
Automação de processos de coleta e limpeza de dados, utilizando Python e SQL, reduzindo o tempo de processamento em 30%.
Colaboração com equipes de TI e engenharia para garantir a integridade e a acessibilidade dos dados necessários para a análise.


Analista de Dados Júnior
InovaTech Consultoria — São Paulo, SP
Janeiro 2016 – Junho 2018

Responsável pela análise exploratória de dados, criação de relatórios e extração de insights para apoiar decisões estratégicas de clientes.
Utilização de SQL para manipulação de grandes bases de dados e otimização de consultas.
Desenvolvimento de scripts em Python para análise estatística e visualização de dados em gráficos interativos.
Suporte na criação de modelos de previsão de vendas e análise de dados financeiros.


Formação Acadêmica
Mestrado em Ciência de Dados
Universidade de São Paulo (USP) — São Paulo, SP
Conclusão: 2021

Bacharelado em Engenharia de Computação
Universidade Estadual de Campinas (UNICAMP) — Campinas, SP
Conclusão: 2015


Certificações
Certificação em Machine Learning – Coursera (2020)
Certificação em Data Science – DataCamp (2019)
Certificação em Análise de Dados com Python – Udemy (2018)
Certificação em Power BI – Microsoft (2017)


Habilidades Técnicas
Linguagens de Programação: Python (pandas, numpy, scikit-learn, tensorflow), SQL, R, MATLAB
Ferramentas de Visualização de Dados: Tableau, Power BI, Matplotlib, Seaborn
Banco de Dados: MySQL, PostgreSQL, MongoDB
Modelos e Algoritmos de Machine Learning: Regressão Linear, Árvores de Decisão, Random Forest, SVM, Redes Neurais
Técnicas Estatísticas: Análise de Regressão, Testes de Hipóteses, Séries Temporais
Plataformas e Ferramentas de Cloud: AWS, Google Cloud, Microsoft Azure


Idiomas
Português: Nativo
Inglês: Avançado (leitura, escrita e conversação)
Espanhol: Intermediário


Projetos Relevantes
Previsão de Demanda em E-commerce: Desenvolvimento de um modelo de previsão de demanda baseado em séries temporais para um cliente no setor de varejo, reduzindo excessos de estoque e aumentando a eficiência da cadeia de suprimentos.
Análise de Sentimentos em Redes Sociais: Criação de um modelo de NLP para análise de sentimentos em posts de redes sociais de uma marca, fornecendo insights acionáveis para a área de marketing sobre percepção de marca.
Otimização de Algoritmos de Recomendação: Melhoria de um sistema de recomendação de produtos baseado em comportamento de compra de usuários, aumentando a taxa de conversão em 12%.
Atividades Complementares
Participação em eventos e workshops sobre Inteligência Artificial e Data Science, como a Data Science Week (2019 e 2020).
Mentor de novos cientistas de dados em plataformas como Kaggle e StackOverflow.
Voluntário em iniciativas de dados abertos e educação tecnológica para jovens em comunidades carentes.


#### Solução

In [ ]:
from langchain_community.document_loaders import TextLoader

In [ ]:
# carregamento dos dados
loader = TextLoader("curriculo_juliano_moreira.txt")
docs = loader.load()

In [ ]:
docs

[Document(metadata={'source': 'curriculo_juliano_moreira.txt'}, page_content='Juliano Moreira\n\n\nResumo Profissional\nCientista de Dados com mais de 5 anos de experiência em análise de dados, machine learning e inteligência artificial. Expertise em transformar grandes volumes de dados em insights estratégicos para tomadas de decisão de negócios. Proficiente em Python, SQL, R, e ferramentas de visualização como Power BI e Tableau. Experiência em trabalhar com modelos preditivos, análise estatística e otimização de processos empresariais. Com forte habilidade de comunicação e apresentação, colaborando estreitamente com equipes multifuncionais para implementar soluções baseadas em dados.\n\n\nExperiência Profissional\nCientista de Dados Sênior\nEmpresa XYZ — São Paulo, SP\nMarço 2022 – Atual\n\nDesenvolvimento e implementação de modelos preditivos de aprendizado de máquina para previsão de demanda e otimização de inventário, resultando em uma redução de 15% nos custos operacionais.\nAná

Pelo comportamento dos dados do currículo, as seções estão separadas por caracteres múltiplos de quebras de linha. Partindo dessa premissa, será usado o CharacterTextSplitter.

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
# splitando o texto por caracteres especiais de quebras de linhas

text_splitter = CharacterTextSplitter(
    separator='\n\n\n',
    chunk_size=800,
    chunk_overlap=100
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)

7

In [ ]:
all_splits[2]

Document(metadata={'source': 'curriculo_juliano_moreira.txt'}, page_content='Cientista de Dados\nTech Solutions Ltda. — São Paulo, SP\nJulho 2018 – Fevereiro 2022\n\nCriação de modelos de machine learning para previsão de churn de clientes, com taxa de sucesso de 80% na antecipação de perdas de clientes.\nAnálise estatística de dados históricos de vendas para identificar tendências e oferecer recomendações de estratégia para a diretoria de vendas.\nDesenvolvimento de algoritmos de recomendação para melhorar a experiência do cliente no e-commerce, resultando em um aumento de 10% na taxa de conversão.\nAutomação de processos de coleta e limpeza de dados, utilizando Python e SQL, reduzindo o tempo de processamento em 30%.\nColaboração com equipes de TI e engenharia para garantir a integridade e a acessibilidade dos dados necessários para a análise.')

In [ ]:
all_splits[4]

Document(metadata={'source': 'curriculo_juliano_moreira.txt'}, page_content='Certificações\nCertificação em Machine Learning – Coursera (2020)\nCertificação em Data Science – DataCamp (2019)\nCertificação em Análise de Dados com Python – Udemy (2018)\nCertificação em Power BI – Microsoft (2017)\n\n\nHabilidades Técnicas\nLinguagens de Programação: Python (pandas, numpy, scikit-learn, tensorflow), SQL, R, MATLAB\nFerramentas de Visualização de Dados: Tableau, Power BI, Matplotlib, Seaborn\nBanco de Dados: MySQL, PostgreSQL, MongoDB\nModelos e Algoritmos de Machine Learning: Regressão Linear, Árvores de Decisão, Random Forest, SVM, Redes Neurais\nTécnicas Estatísticas: Análise de Regressão, Testes de Hipóteses, Séries Temporais\nPlataformas e Ferramentas de Cloud: AWS, Google Cloud, Microsoft Azure')

Gerando o retriever para responder sobre o currículo

In [ ]:
# construcao do banco vetorial
vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

# retriever
retriever = vectorstore.as_retriever(search_type="similarity",
                                     search_kwargs={"k": 3})

In [ ]:
# construção do prompt

system_template_ex1 = """Você é um assistente de RH capaz de analisar currículos dos candidatos e responsável pelas tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template_ex1 = ChatPromptTemplate.from_template(system_template)

In [ ]:
# criando a cadeia usando o llm definido previamente

rag_chain_ex1 = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template_ex1
    | llm
    | StrOutputParser()
)

In [ ]:
def generate_answer(question, rag_model=rag_chain_ex1):
  for chunk in rag_model.stream(question):
    print(chunk, end="", flush=True)

In [ ]:
generate_answer("Juliano Moreira possui conhecimento em Java?")

Não, Juliano Moreira não possui conhecimento em Java. Ele é proficiente em Python, SQL e R.

In [ ]:
generate_answer("Juliano Moreira possui quantos anos de experiência profissional?")

Juliano Moreira possui mais de 5 anos de experiência profissional.

In [ ]:
generate_answer("Juliano Moreira é formado em que curso?")

Juliano Moreira é formado em Bacharelado em Engenharia de Computação e possui Mestrado em Ciência de Dados.